In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

In [2]:
from diffusion.data_loaders.motion_dataset import MotionDataset
dataset = MotionDataset("/home/kenji/Fyp/DeepMimic_mujoco/diffusion/data/motions/humanoid3d_walk.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.1931, -0.2979, -0.0831],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.1812, -0.2662, -0.1258],
         [ 0.0817, -0.0105,  0.8483,  ...,  0.1293, -0.1646, -0.1874],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ...,  0.0799, -0.2361, -0.1045],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0750, -0.2538, -0.1148],
         [ 1.0080,  0.0138,  0.8721,  ...,  0.0762, -0.2713, -0.1198]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02])}

### Model NN

In [3]:
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

savepath = '/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [10]:
from diffusion.diffuser.models.temporal import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
horizon=32
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal.TemporalUnet'>
    cond_dim: 35
    horizon: 32
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion/model_config.pkl

[ models/temporal ] Channel dimensions: [(35, 32), (32, 64), (64, 128), (128, 256)]
[(35, 32), (32, 64), (64, 128), (128, 256)]


In [11]:
from diffusion.diffuser.models.diffusion import GaussianDiffusion

n_timesteps = 20
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, 'diffusion_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion.GaussianDiffusion'>
    action_weight: 5
    clip_denoised: False
    horizon: 32
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 20
    predict_epsilon: False
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion/diffusion_config.pkl



In [12]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion/trainer_config.pkl



In [13]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 32, 35])
x.shape torch.Size([1, 35, 32])
x.shape torch.Size([1, 32, 32])
1
xfinal.shape torch.Size([1, 32, 16])
x.shape torch.Size([1, 64, 16])
2
xfinal.shape torch.Size([1, 64, 8])
x.shape torch.Size([1, 128, 8])
3
xfinal.shape torch.Size([1, 128, 4])
x.shape torch.Size([1, 256, 4])
4
xfinal.shape torch.Size([1, 256, 4])
xt1.shape torch.Size([1, 256, 4])
xt2.shape torch.Size([1, 256, 4])
xt3.shape torch.Size([1, 256, 4])
pop.shape torch.Size([1, 256, 4])
pop.shape torch.Size([1, 128, 8])
pop.shape torch.Size([1, 64, 16])


(tensor([[[ 0.1907, -0.4120,  0.2163,  ...,  0.4205, -0.1087, -0.4488],
          [ 0.0332, -0.0975,  0.7419,  ...,  0.3694,  0.0594, -0.8278],
          [ 0.3381, -0.0609,  0.1372,  ...,  1.0427,  0.3293, -0.3584],
          ...,
          [-0.4308, -0.2090,  0.6057,  ...,  0.1371,  0.0086, -0.4776],
          [-0.0068, -0.0973,  0.2200,  ..., -0.3001,  0.1707, -0.2099],
          [ 0.0530,  0.1302,  0.3731,  ..., -0.0224, -0.3183, -0.0256]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 32, 35]))

In [14]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

In [15]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 1

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 1 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion/state_0.pt
0:   0.1670 | a0_loss:   0.0306 | t:   0.3268
100:   0.0170 | a0_loss:   0.0028 | t:   8.7908
200:   0.0044 | a0_loss:   0.0005 | t:   8.6935
300:   0.0015 | a0_loss:   0.0001 | t:   8.6820
400:   0.0008 | a0_loss:   0.0000 | t:   8.7147
500:   0.0005 | a0_loss:   0.0000 | t:   8.3912
600:   0.0003 | a0_loss:   0.0000 | t:   8.1970
700:   0.0003 | a0_loss:   0.0000 | t:   8.0636
800:   0.0002 | a0_loss:   0.0000 | t:   7.9779
900:   0.0001 | a0_loss:   0.0000 | t:   8.1956
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/walk-motion/state_1.pt


In [16]:
# from diffusion.diffuser.utils import load_diffusion

# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

trainer.load(1)
model = trainer.ema_model

In [17]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions)


                                                                                                    
1 / 20 [###                                                         ]   5% | 4.3 Hz
t : 19 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 20 [######                                                      ]  10% | 8.3 Hz
t : 18 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 20 [#########                                                   ]  15% | 12.1 Hz
t : 17 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                         

In [18]:
sample, sample.trajectories.shape

(Sample(trajectories=tensor([[[-0.0067, -0.0021,  0.8461,  ...,  0.1909, -0.3016, -0.0936],
          [ 0.0473, -0.0040,  0.8615,  ...,  0.1811, -0.2591, -0.1094],
          [ 0.1032, -0.0228,  0.8639,  ...,  0.1325, -0.1521, -0.1582],
          ...,
          [ 0.9593,  0.0071,  0.8669,  ...,  0.0916, -0.2370, -0.0900],
          [ 0.9868,  0.0193,  0.8438,  ...,  0.0842, -0.2468, -0.1166],
          [ 0.9974,  0.0102,  0.8574,  ...,  0.0790, -0.2689, -0.1128]]],
        device='cuda:0'), values=tensor([0.], device='cuda:0'), chains=None),
 torch.Size([1, 32, 35]))

In [19]:
dataset[0], dataset[0].trajectories.shape

(Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.1931, -0.2979, -0.0831],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.1812, -0.2662, -0.1258],
         [ 0.0817, -0.0105,  0.8483,  ...,  0.1293, -0.1646, -0.1874],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ...,  0.0799, -0.2361, -0.1045],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0750, -0.2538, -0.1148],
         [ 1.0080,  0.0138,  0.8721,  ...,  0.0762, -0.2713, -0.1198]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02])}),
 to

In [21]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion {i} saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="motion1.npy")

Motion 0 saved as motion1.npy
